In [5]:
from tqdm.notebook import tqdm as progress
import pandas as pd
import plotly.express as px
from plotly.offline import plot
import dimcli
from dimcli.utils import *
import os, sys, time, json
from IPython.display import Image
from IPython.core.display import HTML 
import csv
from itertools import islice
import ast

In [14]:
#Log into Dimensions
#API key is: 9F8D648F0D7E437CB1736BEBDF007F02
#!pip install dimcli -U --quiet 

print("==\nLogging in..")
# https://digital-science.github.io/dimcli/getting-started.html#authentication
ENDPOINT = "https://app.dimensions.ai"
if 'google.colab' in sys.modules:
    import getpass
    KEY = getpass.getpass(prompt='API Key: ')  
    dimcli.login(key=KEY, endpoint=ENDPOINT)
else:
    KEY = "9F8D648F0D7E437CB1736BEBDF007F02"
    dimcli.login(key=KEY, endpoint=ENDPOINT)
dsl = dimcli.Dsl()

==
Logging in..
Dimcli - Dimensions API Client (v1.2)
Connected to: <https://app.dimensions.ai/api/dsl> - DSL v2.9
Method: manual login


In [7]:
singleRead = {}
file_path = r'\Users\ashle\OneDrive\School\Capstone\dimensionsAI\nameOneIDs.csv'
with open(file_path, 'r') as csv_file:
    # Create a CSV DictReader object
    csv_reader = csv.DictReader(csv_file)

    # Iterate through the rows in the CSV file
    for row in csv_reader:
        # Each row is now a dictionary with column names as keys
        singleRead[row['Name']] = ast.literal_eval(row['ID'])
        
        
first_half_items = list(singleRead.items())
# Create a new dictionary using the first half items
whole_Dict = dict(first_half_items)

print(len(whole_Dict))
print(whole_Dict)


993
{'Yale Nemerson': ['ur.01277425615.41'], 'Elia Ayoub': ['ur.0735577112.30'], 'John Dietschy': ['ur.01025071275.97'], 'Yale Enson': ['ur.0101401022.42'], 'Mehran Goulian': ['ur.0110335427.79'], 'Norman Javitt': ['ur.014367154157.97'], 'Hermes Kontos': ['ur.01365573305.13'], 'Gilbert Levinson': ['ur.063473265.70'], 'Harvey Marver': ['ur.0103350205.47'], 'Harold Neu': ['ur.0770536113.50'], 'Arthur Eisen': ['ur.01205720337.52'], 'Barry Fanburg': ['ur.01371673556.42'], 'Dorothea Zucker-Franklin': ['ur.053601322.36'], 'Nathan Zvaifler': ['ur.0105303412.71'], 'Phyllis Bodel': ['ur.052632615.36'], 'Sidney Cooperband': ['ur.056035721.35'], 'Jacques Kessler': ['ur.0103451421.33'], 'Stanley Korenman': ['ur.054037724.92'], 'Gildon Beall': ['ur.0745507037.13'], 'Neil Cherniack': ['ur.01115513255.09'], 'Blas Frangione': ['ur.01346307176.17'], 'Irwin Freedberg': ['ur.0751444026.49'], 'Thomas Merimee': ['ur.070327605.06'], 'I Samloff': ['ur.047134623.93'], 'Wadi Suki': ['ur.01162021515.51'], 'Shir

In [2]:
search_Var = "abstract + additional_filters + application_number + assignee_cities + assignee_countries + assignee_names + assignee_state_codes + assignees + associated_grant_ids + category_bra + category_for + category_for_2020 + category_hra + category_hrcs_hc + category_hrcs_rac + category_icrp_cso + category_icrp_ct + category_rcdc + claims_amount + current_assignee_names + current_assignees + date + date_inserted + dimensions_url + expiration_date + family_count + family_id + federal_support + filing_date + filing_status + funder_countries + funders + granted_date + granted_year + id + inventor_names + inventors + ipcr + jurisdiction + kind + legal_status + orange_book + original_assignee_names + original_assignees + priority_date + priority_year + publication_date + publication_ids + publication_year + publications + reference_ids + researchers + score + times_cited + title + year"

In [8]:
def single_id(person_id):
    #[basics+categories+extras]
    res3 = dsl.query_iterative(f"""search patents where researchers = "{person_id}" return patents[basics+categories+extras+family_count+family_id+federal_support+filing_date+granted_date+publication_ids]""")
    listDict3 = res3.json['patents']
    number = res3.json['_stats']['total_count']
    print(f'{person_id} id has {number} patents')
    name_list = []
    for patent in listDict3:
        name_list.append(patent)
        

    return(name_list)


#"ur.0706401556.06", Elliot Androphy
#ur.01112411132.31
single_id("ur.0706401556.06")

Starting iteration with limit=1000 skip=0 ...
0-97 / 97 (0.91s)
===
Records extracted: 97


ur.0706401556.06 id has 97 patents


[{'assignee_names': ['UNIV INDIANA RES & TECH CORP',
   'BRIGHAM & WOMENS HOSPITAL INC'],
  'assignees': [{'acronym': 'IUPUI',
    'city_name': 'Indianapolis',
    'country_code': 'US',
    'country_name': 'United States',
    'id': 'grid.257413.6',
    'latitude': 39.776966,
    'linkout': ['http://www.iupui.edu/'],
    'longitude': -86.16858,
    'name': 'Indiana University – Purdue University Indianapolis',
    'state_name': 'Indiana',
    'types': ['Education']},
   {'acronym': 'BWH',
    'city_name': 'Boston',
    'country_code': 'US',
    'country_name': 'United States',
    'id': 'grid.62560.37',
    'latitude': 42.3356,
    'linkout': ['http://www.brighamandwomens.org/'],
    'longitude': -71.106415,
    'name': "Brigham and Women's Hospital",
    'state_name': 'Massachusetts',
    'types': ['Healthcare']}],
  'cpc': ['A61K31/4985',
   'A61K31/501',
   'A61K31/517',
   'A61K45/06',
   'A61P21/00',
   'A61K31/4704',
   'A61K31/4418',
   'A61K31/422',
   'A61K31/437',
   'A61K31/

In [11]:
def findby_Id(nameDict):
    all_Patents= {}
    count=0
    for name, person_id in nameDict.items():
        print(person_id[0])        
        res = dsl.query_iterative(f"""search patents where researchers = "{person_id[0]}" return patents[basics+categories+extras+family_count+family_id+federal_support+filing_date+granted_date+publication_ids]""")
        number = res.json['_stats']['total_count']
        print(f'{person_id[0]} id has {number} patents')
        listDict = res.json['patents']
        name_list = []
        for patent in listDict:
            name_list.append(patent)
        
        all_Patents[name] = name_list
        count+=1
        print(f'{count}. {name}')
  
    return all_Patents
patents = findby_Id(whole_Dict)
# Specify the file name
json_filename = "all_Patents.json"

# Writing the dictionary to a JSON file
with open(json_filename, 'w') as json_file:
    json.dump(patents, json_file, indent=2)

print(f'The data has been written to {json_filename}')

Starting iteration with limit=1000 skip=0 ...


ur.01277425615.41


0-48 / 48 (1.75s)
===
Records extracted: 48
Starting iteration with limit=1000 skip=0 ...


ur.01277425615.41 id has 48 patents
1. Yale Nemerson
ur.0735577112.30


0-1 / 1 (1.98s)
===
Records extracted: 1
Starting iteration with limit=1000 skip=0 ...


...

ur.010546227027.19 id has 0 patents
991. Amir Zarrinpar
ur.01052411656.50


===
Records extracted: 0
Starting iteration with limit=1000 skip=0 ...


ur.01052411656.50 id has 0 patents
992. Elad Ziv
ur.01205077360.52


===
Records extracted: 0


ur.01205077360.52 id has 0 patents
993. Behdad Afzali
The data has been written to all_Patents.json
